In [10]:
import pandas as pd

In [11]:
path = "../data/processed/df_cleaned.csv"
df = pd.read_csv(path)
df

,FOLIO,SEXO,EDAD,edu_inicial,inscrito,nivel_edu,terminado,nt_causa,asesorias,extraord,...,tv_plana,tablet,smartphone,InternetF,sIF_causa,mejora_vida,mejora_trabajo,mejor_decisor,ENT,FACTOR_y
0,2,1.0,13,NaN,1.0,3.0,1.0,0.0,0.0,NaN,...,1.0,1.0,1.0,1.0,0.0,2.0,2.0,2.0,24,809.0
1,7,0.0,8,NaN,1.0,3.0,1.0,0.0,0.0,NaN,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,-1.0,22,722.0
2,14,1.0,11,NaN,1.0,3.0,1.0,0.0,0.0,NaN,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,18,413.0
3,15,0.0,13,NaN,1.0,4.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,2.0,2.0,2.0,18,322.0
4,21,1.0,8,NaN,1.0,3.0,1.0,0.0,1.0,NaN,...,1.0,1.0,1.0,1.0,0.0,2.0,2.0,2.0,26,945.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15836,22700,1.0,6,NaN,1.0,2.0,1.0,0.0,NaN,NaN,...,1.0,0.0,1.0,1.0,0.0,-1.0,1.0,-1.0,6,389.0
15837,22707,1.0,8,NaN,1.0,3.0,1.0,0.0,0.0,NaN,...,1.0,1.0,1.0,1.0,0.0,2.0,2.0,2.0,0,4449.0
15838,22712,1.0,18,NaN,1.0,6.0,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,0.0,2.0,2.0,2.0,16,937.0
15839,22714,0.0,15,NaN,1.0,4.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,2.0,2.0,2.0,21,2035.0


In [12]:
columnsToMap = ["mejora_vida", "mejora_trabajo", "mejor_decisor"]

def drop_columns(df, columns):
    df = df.copy()
    df.drop(columns=columns, inplace=True)
    return df



def count_outliers(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    li = q1 - 1.5 * iqr
    ls = q3 + 1.5 * iqr
    print(li, ls)
    outliers = df[(df[column] < li) | (df[column] > ls)]
    return len(outliers)



def scale_0_1(column):
    q1 = column.quantile(0.25)
    q3 = column.quantile(0.75)
    iqr = q3 - q1

    scaled = (column - q1) / iqr
    return scaled.clip(0, 1)

df.loc[:, "i_confidence_edu"] = df["mejora_vida"] + df["mejora_trabajo"] + df["mejor_decisor"]

print(count_outliers(df, "i_confidence_edu"))

df = drop_columns(df, columnsToMap)

df.loc[:, "i_confidence_edu"] = scale_0_1(df["i_confidence_edu"])


df['i_confidence_edu'].describe()


-1.5 10.5
383


count    15841.000000
mean         0.629969
std          0.421204
min          0.000000
25%          0.000000
50%          0.666667
75%          1.000000
max          1.000000
Name: i_confidence_edu, dtype: float64

In [13]:
drops = df.groupby(["inscrito", "inscrito_actual"]).size().reset_index(name="quantidade")

drops["porcentagem"] = (drops["quantidade"] / drops["quantidade"].sum()) * 100

drops

,inscrito,inscrito_actual,quantidade,porcentagem
0,0.0,0.0,804,5.075437
1,0.0,1.0,219,1.382488
2,1.0,0.0,787,4.968121
3,1.0,1.0,14031,88.573954


In [14]:
df = df.drop(columns=["FOLIO", "x<30", "q_personas", "edu_inicial", "ENT", "FOLIO", "sIF_causa", "ev_nula"])

In [15]:
for i in df.columns:
    print(i,df[i].unique())
    print()

SEXO [1. 0.]

EDAD [13  8 11 14  9 16 15 12 10 17  7 18  6]

inscrito [1. 0.]

nivel_edu [3. 4. 7. 6. 2. 0. 5. 9. 8.]

terminado [ 1. nan  0.]

nt_causa [ 0.  7.  4.  2.  3.  1.  6.  9. 14. 13. 10.  5. 11. 12.]

asesorias [ 0.  1. nan]

extraord [nan  0.  1.]

recursado [nan  0.  1.]

ev_tareas [ 1.  0. nan]

ev_exam [ 1.  0. nan]

ev_digital [ 0.  1. nan]

ev_partcp [ 0.  1. nan]

ev_libreta [ 1.  0. nan]

ev_asist [ 0.  1. nan]

ev_otro [ 0. nan  1.]

inscrito_actual [1. 0.]

nins_causa [ 0. 11.  3.  1.  8.  7. 12.  5. 15.  6.  4. 10. 13. 14.  2.]

entr_diagnost [ 1.  0. nan]

entr_induccion [ 0.  1. nan]

entr_regulariz [ 0.  1. nan]

m_impresos [ 1. nan  0.]

m_pizarra [ 1.  0. nan]

m_audio_vid [ 1.  0. nan]

m_expos [ 1.  0. nan]

m_otro [ 0. nan  1.]

med_plataform [ 1.  0. nan]

med_mail_RS [ 1.  0. nan]

med_clases_v [ 1.  0. nan]

med_presencial [ 0.  1. nan]

t_smartphone [ 0.  1. nan]

t_laptop [ 1.  0. nan]

t_PC [ 0.  1. nan]

t_tablet [ 0.  1. nan]

t_TV [ 0. nan  1.]

t

In [16]:
i_colunas = {    'i_t_tec': ['t_smartphone', 't_laptop', 't_PC', 't_tablet', 't_TV', 't_impresos'],
    'i_mental_h': ['estres', 'depresion', 'desespero', 'antisocial'],
    'i_tec': ['pc', 'laptop', 'tv_plana', 'tablet', 'smartphone'],
    'i_m_trad': ['m_impresos', 'm_pizarra'],
    'i_m_tech': ['m_audio_vid', 'm_expos', 'm_otro'],
    
}

newbinarys = {
    'ev_trad': ['ev_tareas', 'ev_exam'],
    'ev_outro': ['ev_partcp', 'ev_libreta', 'ev_asist', 'ev_otro'],
    'entr_cuidado': ['entr_diagnost', 'entr_induccion', 'entr_regulariz'],
    'apoio': ["t_mama", "t_papa", "t_fam_mujer", "t_fam_hombre", "t_o_mujer", "t_o_hombre"],
    'med_virtual':["med_mail_RS", "med_clases_v", "med_plataform"]
}

for newcol, oldcols in i_colunas.items():
    df[newcol] = scale_0_1(df[oldcols].sum(axis=1))
    df.drop(columns=oldcols, inplace=True)

for newcol, oldcols in newbinarys.items():
    df[newcol] = df[oldcols].any(axis=1).astype(int)
    df.drop(columns=oldcols, inplace=True)



In [17]:
import numpy as np
df.loc[df['nins_causa'] == 0, 'nins_causa'] = np.nan

In [18]:
df.to_csv("../data/processed/df_res.csv", index=False)